# Imports

In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential, Model
import tensorflow.keras.layers as tfl
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [3]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

## Getting X and y

In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:,0].values

In [5]:
test_data = test_df.values
test_data.shape

(28000, 784)

## Getting Train and Test Datasets

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

We'll normalize the data:

In [7]:
X_train=X_train/255.
X_test = X_test/255.
test_data = test_data/255.

In [8]:
X_train.max(), X_test.max(), test_data.max()

(1.0, 1.0, 1.0)

## Converting the Array to Tensorflow Dataset

In [13]:
X_train_ds = tf.data.Dataset.from_tensor_slices(X_train)
y_train_ds = tf.one_hot(y_train, depth=10)
y_train_ds = tf.data.Dataset.from_tensor_slices(y_train_ds)

X_test_ds = tf.data.Dataset.from_tensor_slices(X_test)
y_test_ds = tf.one_hot(y_test, depth=10)
y_test_ds = tf.data.Dataset.from_tensor_slices(y_test_ds)

train_ds = tf.data.Dataset.zip((X_train_ds, y_train_ds))
test_ds = tf.data.Dataset.zip((X_test_ds, y_test_ds))

test_data_ds = tf.data.Dataset.from_tensor_slices(test_data)

In [14]:
batch_size = 64
train_ds = train_ds.batch(batch_size)
test_data_ds = test_data_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

In [15]:
for data, label in train_ds.take(1):
    input_shape = data[0].shape
    output_shape = label[0].shape
    print(data[0].shape)
    print(label[0].shape)

(784,)
(10,)


Excellent!

# Modeling

## Creating Callbacks

We'll be creating two callbacks:
1. `ModelCheckpoint`: This will save the weights of the best model which can be loaded later.
2. `EarlyStopping`: This callback will stop the training if the performance is not improving.

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
def model_checkpoint(name, directory="MNIST"):
    dir = os.path.join(directory, name)
    mch = ModelCheckpoint(
        dir, monitor='val_accuracy', verbose=1, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch',
    )
    return mch

est = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    restore_best_weights=True,
)

## Base Model

We'll start with a base NN model (We already had a base ML model and even that was performing great). The base model here will be a simple feed forward network.

In [12]:
inputs = tfl.Input(shape= input_shape, name="input")
x = tfl.Dense(64, name="dense_1", activation="relu")(inputs)
x = tfl.Dense(128, name="dense_2", activation="relu")(x)
outputs = tfl.Dense(10, activation="softmax", name="output")(x)

model_0 = Model(inputs=inputs, outputs=outputs, name="base_model")

model_0.summary()

Model: "base_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________


In [13]:
model_0.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=['accuracy'],
)

In [14]:
EPOCHS = 10
history = model_0.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint("model_0")],
)

Epoch 1/10


2022-06-23 11:29:02.662595: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


591/591 [==============================] - 4s 4ms/step - loss: 0.3787 - accuracy: 0.8933 - val_loss: 0.2172 - val_accuracy: 0.9336

Epoch 00001: val_accuracy improved from -inf to 0.93357, saving model to MNIST/model_0


2022-06-23 11:29:06.070469: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/10
591/591 [==============================] - 2s 3ms/step - loss: 0.1563 - accuracy: 0.9538 - val_loss: 0.1583 - val_accuracy: 0.9521

Epoch 00002: val_accuracy improved from 0.93357 to 0.95214, saving model to MNIST/model_0
Epoch 3/10
591/591 [==============================] - 2s 3ms/step - loss: 0.1090 - accuracy: 0.9679 - val_loss: 0.1301 - val_accuracy: 0.9600

Epoch 00003: val_accuracy improved from 0.95214 to 0.96000, saving model to MNIST/model_0
Epoch 4/10
591/591 [==============================] - 2s 3ms/step - loss: 0.0821 - accuracy: 0.9758 - val_loss: 0.1260 - val_accuracy: 0.9610

Epoch 00004: val_accuracy improved from 0.96000 to 0.96095, saving model to MNIST/model_0
Epoch 5/10
591/591 [==============================] - 2s 3ms/step - loss: 0.0637 - accuracy: 0.9812 - val_loss: 0.1251 - val_accuracy: 0.9610

Epoch 00005: val_accuracy did not improve from 0.96095
Epoch 6/10
591/591 [==============================] - 2s 3ms/step - loss: 0.0500 - accuracy: 0.9858 - v

In [17]:
model_0.evaluate(test_ds), model_0.evaluate(train_ds)

591/591 [==============================] - 1s 2ms/step - loss: 0.0297 - accuracy: 0.9897


([0.12471525371074677, 0.9697619080543518],
 [0.029692815616726875, 0.9896560907363892])

We see that even this simple a model performs excellent.

## Model_1

This will be a shallow CNN. We'll reshape the one column data into $28\times 28\times 1$ so that a CNN can work on it.

In [21]:
inputs = tfl.Input(shape= input_shape, name="input")
x = tfl.Reshape((28, 28, 1), name="reshape")(inputs)
x = tfl.Conv2D(32, 5, name="conv_1")(x)
x = tfl.Conv2D(64, 5, name="conv_2")(x)
x = tfl.GlobalAveragePooling2D(name="global_avg_pooling")(x)
x = tfl.Dense(64, activation="relu")(x)
outputs = tfl.Dense(10, activation="softmax", name="output")(x)
model_1 = Model(inputs=inputs, outputs=outputs, name="simple_conv_model")

model_1.summary()

Model: "simple_conv_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv_2 (Conv2D)              (None, 20, 20, 64)        51264     
_________________________________________________________________
global_avg_pooling (GlobalAv (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 10)          

In [22]:
EPOCHS = 10
model_1.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=['accuracy'],
)


history_1 = model_1.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
#     callbacks=[model_checkpoint(model_1.name)],
)

Epoch 1/10
591/591 [==============================] - 3s 4ms/step - loss: 1.4886 - accuracy: 0.4763 - val_loss: 1.2349 - val_accuracy: 0.5402
Epoch 2/10
591/591 [==============================] - 2s 4ms/step - loss: 1.0519 - accuracy: 0.6377 - val_loss: 1.0052 - val_accuracy: 0.6474
Epoch 3/10
591/591 [==============================] - 3s 5ms/step - loss: 0.9444 - accuracy: 0.6773 - val_loss: 0.9138 - val_accuracy: 0.6926
Epoch 4/10
591/591 [==============================] - 2s 4ms/step - loss: 0.8818 - accuracy: 0.7024 - val_loss: 0.8471 - val_accuracy: 0.7131
Epoch 5/10
591/591 [==============================] - 2s 4ms/step - loss: 0.8292 - accuracy: 0.7206 - val_loss: 0.7912 - val_accuracy: 0.7336
Epoch 6/10
591/591 [==============================] - 2s 4ms/step - loss: 0.7846 - accuracy: 0.7325 - val_loss: 0.7520 - val_accuracy: 0.7517
Epoch 7/10
591/591 [==============================] - 3s 4ms/step - loss: 0.7496 - accuracy: 0.7459 - val_loss: 0.7190 - val_accuracy: 0.7626
Epoch 

In [23]:
model_1.evaluate(test_ds), model_1.evaluate(train_ds)

591/591 [==============================] - 2s 3ms/step - loss: 0.6592 - accuracy: 0.7747


([0.6770902276039124, 0.7752380967140198],
 [0.6592001914978027, 0.7746560573577881])

It seems we need to go deeper. This will be our next model. But before that let's try the same model as `model_1` but `GlobalAveragePooling2D` replaced with `GlobalMaxPooling2D`.

In [24]:
inputs = tfl.Input(shape= input_shape, name="input")
x = tfl.Reshape((28, 28, 1), name="reshape")(inputs)
x = tfl.Conv2D(32, 5, name="conv_1")(x)
x = tfl.Conv2D(64, 5, name="conv_2")(x)
x = tfl.GlobalMaxPooling2D(name="global_avg_pooling")(x)
x = tfl.Dense(64, activation="relu")(x)
outputs = tfl.Dense(10, activation="softmax", name="output")(x)
model_1 = Model(inputs=inputs, outputs=outputs, name="simple_conv_model")


EPOCHS = 10
model_1.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=['accuracy'],
)

history_1 = model_1.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
#     callbacks=[model_checkpoint(model_1.name)],
)

Epoch 1/10
591/591 [==============================] - 3s 4ms/step - loss: 0.4270 - accuracy: 0.8774 - val_loss: 0.1236 - val_accuracy: 0.9643
Epoch 2/10
591/591 [==============================] - 2s 4ms/step - loss: 0.1243 - accuracy: 0.9613 - val_loss: 0.1028 - val_accuracy: 0.9712
Epoch 3/10
591/591 [==============================] - 3s 4ms/step - loss: 0.1016 - accuracy: 0.9672 - val_loss: 0.0997 - val_accuracy: 0.9698
Epoch 4/10
591/591 [==============================] - 2s 4ms/step - loss: 0.0901 - accuracy: 0.9716 - val_loss: 0.0966 - val_accuracy: 0.9705
Epoch 5/10
591/591 [==============================] - 2s 4ms/step - loss: 0.0815 - accuracy: 0.9742 - val_loss: 0.1004 - val_accuracy: 0.9710
Epoch 6/10
591/591 [==============================] - 3s 4ms/step - loss: 0.0750 - accuracy: 0.9765 - val_loss: 0.1041 - val_accuracy: 0.9681
Epoch 7/10
591/591 [==============================] - 2s 4ms/step - loss: 0.0705 - accuracy: 0.9779 - val_loss: 0.0996 - val_accuracy: 0.9700
Epoch 

In [25]:
model_1.evaluate(test_ds), model_1.evaluate(train_ds)

591/591 [==============================] - 2s 3ms/step - loss: 0.0768 - accuracy: 0.9745


([0.1085703894495964, 0.9671428799629211],
 [0.07677440345287323, 0.9744709134101868])

OMG! Just one change is resulting in a massive improvement. Well, it seems that from now one we should use be using `GlobalMaxPooling2D` instead of `GlobalAveragePooling2D`.

## Model 2

In [46]:
inputs = tfl.Input(shape= input_shape, name="input")
x = tfl.Reshape((28, 28, 1), name="reshape")(inputs)
x = tfl.Conv2D(32, 3, name="conv_11", activation='relu')(x)
x = tfl.Conv2D(64, 3, name="conv_12", activation='relu')(x)
x = tfl.MaxPool2D(pool_size=(2, 2), name="maxpool_1")(x)
x = tfl.BatchNormalization(axis=-1, name="batchnorm_1")(x)

x = tfl.Conv2D(128, 3, name="conv_21", activation='relu')(x)
x = tfl.Conv2D(256, 3, name="conv_22", activation='relu')(x)
x = tfl.MaxPool2D(pool_size=(2, 2), name="maxpool_2")(x)
x = tfl.Dropout(0.25, name="dropout_21")(x)
x = tfl.BatchNormalization(axis=-1, name="batchnorm_2")(x)

x = tfl.Conv2D(512, 2, name="conv_31", activation='relu')(x)
x = tfl.Conv2D(1024, 2, name="conv_32", activation='relu')(x)
# x = tfl.Conv2D(1024, 2, name="conv_33")(x)
# x = tfl.MaxPool2D(pool_size=(2, 2), name="maxpool3")(x)
# x = tfl.BatchNormalization(axis=-1, name="batchnorm3")(x)

# x = tfl.Conv2D(512, 2, name="conv_31")(x)
# x = tfl.Conv2D(1024, 2, name="conv_32")(x)
# x = tfl.MaxPool2D(pool_size=(2, 2), name="maxpool3")(x)


x = tfl.GlobalMaxPooling2D(name="global_avg_pooling")(x)
# x = tfl.Flatten(name="flatten")(x)
# x = tfl.BatchNormalization(axis=-1, name="batchnorm3")(x)
# x = tfl.Dense(64, activation="relu", name="dense_1")(x)
x = tfl.Dense(128, activation="relu", name="dense_1")(x)
x = tfl.Dropout(0.25, name="dropout_1")(x)
x = tfl.Dense(256, activation="relu", name="dense_2")(x)
x = tfl.Dropout(0.4, name="dropout_2")(x)
outputs = tfl.Dense(10, activation="softmax", name="output")(x)
model_2 = Model(inputs=inputs, outputs=outputs, name="deep_conv_model")

model_2.summary()

Model: "deep_conv_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv_11 (Conv2D)             (None, 26, 26, 32)        320       
_________________________________________________________________
conv_12 (Conv2D)             (None, 24, 24, 64)        18496     
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 12, 12, 64)        0         
_________________________________________________________________
batchnorm_1 (BatchNormalizat (None, 12, 12, 64)        256       
_________________________________________________________________
conv_21 (Conv2D)             (None, 10, 10, 128)   

In [47]:
EPOCHS = 10
model_2.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=['accuracy'],
)


history_2 = model_2.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
#     callbacks=[model_checkpoint(model_2.name)],
)

Epoch 1/10
591/591 [==============================] - 81s 10ms/step - loss: 0.2534 - accuracy: 0.9238 - val_loss: 0.0738 - val_accuracy: 0.9786
Epoch 2/10
591/591 [==============================] - 6s 10ms/step - loss: 0.0845 - accuracy: 0.9794 - val_loss: 0.0682 - val_accuracy: 0.9855
Epoch 3/10
591/591 [==============================] - 6s 10ms/step - loss: 0.0617 - accuracy: 0.9852 - val_loss: 0.0518 - val_accuracy: 0.9881
Epoch 4/10
591/591 [==============================] - 6s 10ms/step - loss: 0.0558 - accuracy: 0.9859 - val_loss: 0.0921 - val_accuracy: 0.9826
Epoch 5/10
591/591 [==============================] - 6s 10ms/step - loss: 0.0461 - accuracy: 0.9892 - val_loss: 0.0508 - val_accuracy: 0.9874
Epoch 6/10
591/591 [==============================] - 6s 10ms/step - loss: 0.0413 - accuracy: 0.9897 - val_loss: 0.0489 - val_accuracy: 0.9876
Epoch 7/10
591/591 [==============================] - 6s 10ms/step - loss: 0.0361 - accuracy: 0.9909 - val_loss: 0.0658 - val_accuracy: 0.985

Let's train for some more epochs.

In [48]:
EPOCHS = 15

history_2 = model_2.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint(model_2.name)],
)

Epoch 1/15
591/591 [==============================] - 6s 10ms/step - loss: 0.0268 - accuracy: 0.9936 - val_loss: 0.0560 - val_accuracy: 0.9881

Epoch 00001: val_accuracy improved from -inf to 0.98810, saving model to MNIST/deep_conv_model
Epoch 2/15
591/591 [==============================] - 6s 10ms/step - loss: 0.0300 - accuracy: 0.9928 - val_loss: 0.0582 - val_accuracy: 0.9900

Epoch 00002: val_accuracy improved from 0.98810 to 0.99000, saving model to MNIST/deep_conv_model
Epoch 3/15
591/591 [==============================] - 6s 10ms/step - loss: 0.0281 - accuracy: 0.9932 - val_loss: 0.0382 - val_accuracy: 0.9914

Epoch 00003: val_accuracy improved from 0.99000 to 0.99143, saving model to MNIST/deep_conv_model
Epoch 4/15
591/591 [==============================] - 6s 10ms/step - loss: 0.0243 - accuracy: 0.9938 - val_loss: 0.0429 - val_accuracy: 0.9907

Epoch 00004: val_accuracy did not improve from 0.99143
Epoch 5/15
591/591 [==============================] - 6s 10ms/step - loss: 0.0

In [49]:
model_2.evaluate(test_ds), model_2.evaluate(train_ds)

591/591 [==============================] - 3s 4ms/step - loss: 0.0118 - accuracy: 0.9977


([0.06591577082872391, 0.9902380704879761],
 [0.011813807301223278, 0.9976984262466431])

In [50]:
best_model_2 = tf.keras.models.load_model("MNIST/deep_conv_model")

best_model_2.evaluate(test_ds), best_model_2.evaluate(train_ds)

591/591 [==============================] - 3s 4ms/step - loss: 0.0098 - accuracy: 0.9976


([0.048807624727487564, 0.9916666746139526],
 [0.009835764765739441, 0.9976190328598022])

## Making Predictions

In [25]:
y_pred = model_2.predict(test_data_ds)
y_pred = np.argmax(y_pred, axis = -1)
submission = pd.DataFrame({'ImageId': range(1, test_data.shape[0]+1), 'Label': y_pred})
submission.groupby('Label').count()

In [26]:
y_pred

array([2, 0, 9, ..., 3, 9, 2])

## Final Model

In [51]:
model_3 = Sequential(name="final_model")
model_3.add(tfl.Reshape((28, 28, 1), name="reshape", input_shape=input_shape))
model_3.add(tfl.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1),
             padding='same')),
model_3.add(tfl.Conv2D(32, kernel_size=(3, 3), activation='relu',
                padding='same')),
model_3.add(tfl.AveragePooling2D(pool_size=(2, 2)))
model_3.add(tfl.BatchNormalization())
model_3.add(tfl.Dropout(0.3))
model_3.add(tfl.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model_3.add(tfl.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model_3.add(tfl.AveragePooling2D(pool_size=(2, 2)))
model_3.add(tfl.BatchNormalization())
model_3.add(tfl.Dropout(0.3))
model_3.add(tfl.Conv2D(128, kernel_size=(3, 3), activation='relu' , padding='same'))
model_3.add(tfl.Conv2D(128, kernel_size=(3, 3), activation='relu' , padding='same'))
model_3.add(tfl.AveragePooling2D(pool_size=(2, 2)))
model_3.add(tfl.BatchNormalization())

model_3.add(tfl.Flatten())
model_3.add(tfl.Dropout(0.35))
model_3.add(tfl.Dense(128, activation='relu'))
model_3.add(tfl.BatchNormalization())
model_3.add(tfl.Dense(10, activation='softmax'))

model_3.compile(optimizer='adam', loss='categorical_crossentropy', 
    metrics=['accuracy'])

model_3.summary()

Model: "final_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
average_pooling2d_9 (Average (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 14, 14, 64)        

In [52]:
history_3 = model_3.fit(train_ds, epochs=10, validation_data=test_ds,
#         callbacks=[model_checkpoint(model_3.name)]
                      )

Epoch 1/10
591/591 [==============================] - 5s 8ms/step - loss: 0.1584 - accuracy: 0.9515 - val_loss: 0.0717 - val_accuracy: 0.9764
Epoch 2/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0601 - accuracy: 0.9818 - val_loss: 0.0414 - val_accuracy: 0.9874
Epoch 3/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0469 - accuracy: 0.9853 - val_loss: 0.0301 - val_accuracy: 0.9907
Epoch 4/10
591/591 [==============================] - 4s 7ms/step - loss: 0.0385 - accuracy: 0.9877 - val_loss: 0.0257 - val_accuracy: 0.9924
Epoch 5/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0330 - accuracy: 0.9893 - val_loss: 0.0319 - val_accuracy: 0.9905
Epoch 6/10
591/591 [==============================] - 4s 6ms/step - loss: 0.0307 - accuracy: 0.9905 - val_loss: 0.0287 - val_accuracy: 0.9902
Epoch 7/10
591/591 [==============================] - 4s 7ms/step - loss: 0.0271 - accuracy: 0.9912 - val_loss: 0.0336 - val_accuracy: 0.9907
Epoch 

In [53]:
history_3 = model_3.fit(train_ds, epochs=30, validation_data=test_ds,
        callbacks=[model_checkpoint(model_3.name)]
                      )

Epoch 1/30
591/591 [==============================] - 4s 6ms/step - loss: 0.0201 - accuracy: 0.9937 - val_loss: 0.0244 - val_accuracy: 0.9917

Epoch 00001: val_accuracy improved from -inf to 0.99167, saving model to MNIST/final_model
Epoch 2/30
591/591 [==============================] - 4s 7ms/step - loss: 0.0170 - accuracy: 0.9940 - val_loss: 0.0275 - val_accuracy: 0.9914

Epoch 00002: val_accuracy did not improve from 0.99167
Epoch 3/30
591/591 [==============================] - 4s 6ms/step - loss: 0.0164 - accuracy: 0.9950 - val_loss: 0.0282 - val_accuracy: 0.9910

Epoch 00003: val_accuracy did not improve from 0.99167
Epoch 4/30
591/591 [==============================] - 4s 7ms/step - loss: 0.0153 - accuracy: 0.9950 - val_loss: 0.0279 - val_accuracy: 0.9929

Epoch 00004: val_accuracy improved from 0.99167 to 0.99286, saving model to MNIST/final_model
Epoch 5/30
591/591 [==============================] - 4s 6ms/step - loss: 0.0143 - accuracy: 0.9952 - val_loss: 0.0205 - val_accuracy

In [54]:
model_3.evaluate(test_ds), model_3.evaluate(train_ds)

591/591 [==============================] - 2s 3ms/step - loss: 9.1685e-04 - accuracy: 0.9997


([0.02114895172417164, 0.9947618842124939],
 [0.0009168467367999256, 0.9997090101242065])

### Making Submission File

In [55]:
y_pred = model_2.predict(test_data_ds)
y_pred = np.argmax(y_pred, axis = -1)
submission = pd.DataFrame({'ImageId': range(1, test_data.shape[0]+1), 'Label': y_pred})
submission.groupby('Label').count()

,ImageId
Label,
0,2759
1,3200
2,2806
3,2776
4,2753
5,2519
6,2735
7,2880
8,2808


In [58]:
submission.to_csv("submission.csv", index = False)

In [59]:
from IPython.display import FileLink
FileLink("submission.csv")

/kaggle/working/submission.csv